# Load in FinSen dataset

In [ ]:
import pandas as pd

In [ ]:
# https://github.com/EagleAdelaide/FinSen_Dataset
FinSen = pd.read_csv('/content/drive/MyDrive/INST762/FinSen_US_Categorized_Timestamp.csv')

In [ ]:
FinSen["timestamp"] = pd.to_datetime(FinSen["Time"], dayfirst=True)
FinSen.drop(columns=['Time'], inplace=True)

In [39]:
FinSen.head()

,Title,Tag,Content,Date,SentimentLabel,SentimentConfidenceScore,positive_score,negative_score,neutral_score
0,Visa Hits 24-week High,stocks,Visa Hits 24-week HighUnited States stocksVisa...,2023-07-14,positive,0.753741,0.435567,0.384917,0.179516
1,Amazon Hits 43-week High,stocks,Amazon Hits 43-week HighUnited States stocksAm...,2023-07-14,positive,0.541875,0.262671,0.704302,0.033027
2,Visa Hits 24-week High,stocks,Visa Hits 24-week HighUnited States stocksVisa...,2023-07-14,positive,0.753741,0.013053,0.970463,0.016484
3,Amazon Hits 43-week High,stocks,Amazon Hits 43-week HighUnited States stocksAm...,2023-07-14,positive,0.541875,0.427173,0.047533,0.525293
4,US Futures Steady Ahead of Key Inflation Data,stock market,US Futures Steady Ahead of Key Inflation DataU...,2023-07-13,positive,0.435567,0.023399,0.932868,0.043733


# Subset dataset to only stock-related articles

In [ ]:
FinSen['Tag'].value_counts().head()

,count
Tag,
Stock Market,3632
Earnings,2731
Currency,1578
Government Bond 10Y,780
Crude Oil Stocks Change,372


In [ ]:
unique_tags = FinSen['Tag'].unique()
unique_tags = [tag.lower() for tag in unique_tags]
multiword_tags = [tag for tag in unique_tags if ' ' in tag]
singleword_tags = [tag for tag in unique_tags if ' ' not in tag]
len(unique_tags), len(multiword_tags), len(singleword_tags)

(144, 127, 17)

In [ ]:
unique_tags = list(set(unique_tags))

In [ ]:
stock_tags = [tag.lower() for tag in unique_tags if 'stock' in tag]
len(stock_tags), stock_tags

(8,
 ['stock market',
  'stocks',
  'heating oil stocks',
  'distillate stocks',
  'crude oil stocks change',
  'gasoline stocks change',
  'natural gas stocks change',
  'api crude oil stock change'])

In [ ]:
FinSenStockArticles = FinSen[FinSen['Tag'].isin(stock_tags)]

# Generate sentiment scores based on stock article titles

In [40]:
# https://www.quantconnect.com/docs/v2/writing-algorithms/machine-learning/hugging-face/popular-models/finbert
# https://huggingface.co/ProsusAI/finbert
# https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer
# https://github.com/ProsusAI/finBERT/tree/master
# https://docs.pytorch.org/docs/stable/generated/torch.nn.functional.softmax.html

from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")
model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert")
model.eval()

def get_sentiments(text):
    """
    Analyzes the sentiment of financial text and returns the positive, negative, and neutral scores.

    Args:
        text (str): The financial text to analyze.

    Returns:
        list containing the positive, negative, and neutral scores.
    """
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        sents = torch.nn.functional.softmax(outputs.logits, dim=1).cpu().numpy().flatten()
    return sents

In [42]:
sents = FinSenStockArticles['Title'][0:5].apply(get_sentiments)

In [43]:
sents_df = pd.DataFrame(sents.tolist(), columns=['positive_score', 'negative_score', 'neutral_score'])

In [44]:
sents_df.head()

,positive_score,negative_score,neutral_score
0,0.840223,0.039225,0.120552
1,0.012411,0.956066,0.031524
2,0.051733,0.758506,0.189761
3,0.496655,0.083024,0.420321
4,0.422558,0.191542,0.385900


In [ ]:
FinStockArticlesWithSentiment = pd.concat([FinSenStockArticles, sents_df], axis=1)

# Load in SP500 data from marketwatch

In [ ]:
SP500_1 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX.csv')
SP500_2 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (1).csv')
SP500_3 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (2).csv')
SP500_4 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (3).csv')
SP500_5 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (4).csv')
SP500_6 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (5).csv')
SP500_7 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (6).csv')
SP500_8 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (7).csv')
SP500_9 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (8).csv')
SP500_10 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (9).csv')
SP500_11 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (10).csv')
SP500_12 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (11).csv')
SP500_13 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (12).csv')
SP500_14 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (13).csv')
SP500_15 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (14).csv')
SP500_16 = pd.read_csv('/content/drive/MyDrive/INST762/Download Data - INDEX_US_S&P US_SPX (15).csv')

## Apply transformations to each to prepare for concatenation

In [ ]:
SP500 = pd.concat([SP500_1, SP500_2, SP500_3, SP500_4, SP500_5, SP500_6, SP500_7, SP500_8, SP500_9, SP500_10, SP500_11, SP500_12, SP500_13, SP500_14, SP500_15, SP500_16])

In [ ]:
FinSen = pd.read_csv('/content/drive/MyDrive/FinStockArticlesWithSentiment.csv')

In [ ]:
SP500 = pd.read_csv('/content/drive/MyDrive/SP500.csv')

In [ ]:
cols = ['Open', 'High', 'Low', 'Close']
SP500[cols] = SP500[cols].replace({',': ''}, regex=True).apply(pd.to_numeric)

In [ ]:
cols = ['SentimentConfidenceScore', 'positive_score', 'negative_score', 'neutral_score']
FinSen[cols] = FinSen[cols].apply(pd.to_numeric)

In [ ]:
FinSen.rename(columns={'timestamp': 'Date'}, inplace=True)
FinSen['Date'] = pd.to_datetime(FinSen['Date'])

In [ ]:
SP500['Date'] = pd.to_datetime(SP500['Date'])

### Aggregating by month: finding the mean, median, range for all values

In [ ]:
FinSenMonthMeans = FinSen.groupby(pd.Grouper(key='Date', freq='ME'))[['SentimentConfidenceScore', 'positive_score', 'negative_score', 'neutral_score']].mean().dropna()
FinSenMonthMedians = FinSen.groupby(pd.Grouper(key='Date', freq='ME'))[['SentimentConfidenceScore', 'positive_score', 'negative_score', 'neutral_score']].median().dropna()
FinSenMonthMaxes = FinSen.groupby(pd.Grouper(key='Date', freq='ME'))[['SentimentConfidenceScore', 'positive_score', 'negative_score', 'neutral_score']].max().dropna()
FinSenMonthMins = FinSen.groupby(pd.Grouper(key='Date', freq='ME'))[['SentimentConfidenceScore', 'positive_score', 'negative_score', 'neutral_score']].min().dropna()
FinSenMonthRanges = FinSenMonthMaxes - FinSenMonthMins
SP500MonthMeans = SP500.groupby(pd.Grouper(key='Date', freq='ME'))[['Open', 'High', 'Low', 'Close']].mean().dropna()
SP500MonthMedians = SP500.groupby(pd.Grouper(key='Date', freq='ME'))[['Open', 'High', 'Low', 'Close']].median().dropna()
SP500MonthMaxes = SP500.groupby(pd.Grouper(key='Date', freq='ME'))[['Open', 'High', 'Low', 'Close']].max().dropna()
SP500MonthMins = SP500.groupby(pd.Grouper(key='Date', freq='ME'))[['Open', 'High', 'Low', 'Close']].min().dropna()
SP500MonthRanges = SP500MonthMaxes - SP500MonthMins

### Transforming dataset for usability

In [ ]:
SP500cols = ['Open', 'High', 'Low', 'Close']
FinSencols = ['SentimentConfidenceScore', 'positive_score', 'negative_score', 'neutral_score']

In [ ]:
for col in FinSencols:
  FinSenMonthMeans.rename(columns={col: f'{col}_mean'}, inplace=True)
  FinSenMonthMedians.rename(columns={col: f'{col}_median'}, inplace=True)
  FinSenMonthRanges.rename(columns={col: f'{col}_range'}, inplace=True)

In [ ]:
for col in SP500cols:
  SP500MonthMeans.rename(columns={col: f'{col}_mean'}, inplace=True)
  SP500MonthMedians.rename(columns={col: f'{col}_median'}, inplace=True)
  SP500MonthRanges.rename(columns={col: f'{col}_range'}, inplace=True)

In [ ]:
MonthMeans = pd.merge(SP500MonthMeans, FinSenMonthMeans, on='Date')

In [ ]:
MonthMedians = pd.merge(SP500MonthMedians, FinSenMonthMedians, on='Date')

In [ ]:
MonthRanges = pd.merge(SP500MonthRanges, FinSenMonthRanges, on='Date')

In [ ]:
MonthMeansMedians = pd.merge(MonthMeans, MonthMedians, on='Date')

In [ ]:
MonthStats = pd.merge(MonthMeansMedians, MonthRanges, on='Date')

In [ ]:
MonthStats.reset_index(inplace=True)

In [ ]:
MonthStats.columns

Index(['Date', 'Open_mean', 'High_mean', 'Low_mean', 'Close_mean',
       'SentimentConfidenceScore_mean', 'positive_score_mean',
       'negative_score_mean', 'neutral_score_mean', 'Open_median',
       'High_median', 'Low_median', 'Close_median',
       'SentimentConfidenceScore_median', 'positive_score_median',
       'negative_score_median', 'neutral_score_median', 'Open_range',
       'High_range', 'Low_range', 'Close_range',
       'SentimentConfidenceScore_range', 'positive_score_range',
       'negative_score_range', 'neutral_score_range'],
      dtype='object')

In [ ]:
MonthStats.head()

,Date,Open_mean,High_mean,Low_mean,Close_mean,SentimentConfidenceScore_mean,positive_score_mean,negative_score_mean,neutral_score_mean,Open_median,...,negative_score_median,neutral_score_median,Open_range,High_range,Low_range,Close_range,SentimentConfidenceScore_range,positive_score_range,negative_score_range,neutral_score_range
0,2022-01-31,4585.263000,4619.576000,4528.042000,4573.815500,0.756258,0.157939,0.752225,0.089836,4635.115,...,0.914991,0.052981,468.32,407.61,551.65,470.05,0.610559,0.686553,0.940359,0.267110
1,2022-02-28,4436.878947,4473.607368,4392.072632,4435.980526,0.779914,0.283549,0.512375,0.204075,4456.060,...,0.471314,0.074494,410.62,300.58,432.35,363.88,0.593445,0.914829,0.953647,0.852308
2,2022-03-31,4388.294348,4424.881739,4351.570000,4391.265217,0.694431,0.312590,0.601066,0.086343,4363.140,...,0.809917,0.049874,435.38,389.73,431.79,460.90,0.591100,0.920403,0.946491,0.320288
3,2022-04-30,4409.360500,4439.264500,4361.126500,4391.296000,0.791007,0.363222,0.540196,0.096582,4443.355,...,0.817529,0.052641,385.93,352.74,414.93,450.71,0.621523,0.937602,0.939545,0.404270
4,2022-05-31,4037.771429,4082.188095,3986.214286,4040.360000,0.745236,0.272307,0.554348,0.173345,4035.180,...,0.525620,0.058519,371.43,364.24,338.59,399.38,0.577713,0.945400,0.954322,0.762163


In [ ]:
MonthStats.to_csv('MonthStats.csv', index=False)